# PREDICCIÓN DE VENTAS 
## Este script toma las ventas históricas y predice los próximos 90 días

### Carga de Librerías y Datos

In [28]:
import pandas as pd
import numpy as np
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
import warnings
warnings.filterwarnings('ignore')

In [29]:
df = pd.read_csv('Datos/ventas_dataset_sucio.csv')

In [30]:
print(f"✓ Archivo leído: {len(df)} registros")
print(f"✓ Columnas: {list(df.columns)}\n")

✓ Archivo leído: 18433 registros
✓ Columnas: ['Order_ID', 'Fecha', 'Categoria', 'Producto', 'Codigo_Producto', 'Canal', 'Region', 'Precio_Unitario', 'Costo_Unitario', 'Descuento', 'Cantidad', 'Ventas', 'Cliente_ID']



In [31]:
df.head()

,Order_ID,Fecha,Categoria,Producto,Codigo_Producto,Canal,Region,Precio_Unitario,Costo_Unitario,Descuento,Cantidad,Ventas,Cliente_ID
0,ORD-41186,2023-06-22,Muebles,Archivador Metálico,PROD-810,Tienda Física,Centro,$125.93,287.87,0.05,5,629.64,CLI-9217
1,ORD-87032,29/11/2023,Tecnología,Monitor LG,PROD-812,Tienda Física,Sur,78.58,271.75,0.20,5,392.92,CLI-2308
2,ORD-44311,2022-11-10,Muebles,Silla Ergonómica,PROD-956,Distribuidor,Oeste,60.41,248.03,NaN,5,302.06,CLI-3580
3,ORD-15526,21/02/2023,Tecnología,Teclado Mecánico,PROD-513,Distribuidor,Norte,$119.82,262.28,0.20,3,359.46,CLI-4570
4,ORD-61931,2022-04-14,Tecnología,Laptop Dell,PROD-970,Tienda Física,Este,373.24,837.17,0.15,2,746.47,CLI-9353


### Limpieza de datos

In [32]:
# 1. LIMPIAR COLUMNA VENTAS
df['Ventas'] = df['Ventas'].astype(str).str.replace('$', '', regex = False)
df['Ventas'] = df['Ventas'].str.replace(',', '', regex = False)
df['Ventas'] = df['Ventas'].str.strip()
df['Ventas'] = pd.to_numeric(df['Ventas'], errors = 'coerce')

In [33]:
# 2. LIMPIAR FECHAS
def limpiar_fecha(fecha_str):
    
    if pd.isna(fecha_str):
        return None
    
    fecha_str = str(fecha_str).strip()
    
    formatos = [
        '%Y-%m-%d',
        '%d/%m/%Y',
        '%m-%d-%Y',
        '%Y/%m/%d',
        '%d-%m-%Y'
    ]
    
    for formato in formatos:
        try:
            return pd.to_datetime(fecha_str, format = formato)
        except:
            continue
    
    return None

In [34]:
df['Fecha_Limpia'] = df['Fecha'].apply(limpiar_fecha)

In [35]:
df.head()

,Order_ID,Fecha,Categoria,Producto,Codigo_Producto,Canal,Region,Precio_Unitario,Costo_Unitario,Descuento,Cantidad,Ventas,Cliente_ID,Fecha_Limpia
0,ORD-41186,2023-06-22,Muebles,Archivador Metálico,PROD-810,Tienda Física,Centro,$125.93,287.87,0.05,5,629.64,CLI-9217,2023-06-22
1,ORD-87032,29/11/2023,Tecnología,Monitor LG,PROD-812,Tienda Física,Sur,78.58,271.75,0.20,5,392.92,CLI-2308,2023-11-29
2,ORD-44311,2022-11-10,Muebles,Silla Ergonómica,PROD-956,Distribuidor,Oeste,60.41,248.03,NaN,5,302.06,CLI-3580,2022-11-10
3,ORD-15526,21/02/2023,Tecnología,Teclado Mecánico,PROD-513,Distribuidor,Norte,$119.82,262.28,0.20,3,359.46,CLI-4570,2023-02-21
4,ORD-61931,2022-04-14,Tecnología,Laptop Dell,PROD-970,Tienda Física,Este,373.24,837.17,0.15,2,746.47,CLI-9353,2022-04-14


In [36]:
# 3. FILTROS DE CALIDAD
registros_inicial = len(df)
registros_inicial

18433

In [37]:
# Quitar registros sin fecha válida
df = df[df['Fecha_Limpia'].notna()].copy()
print(f"• Removidos sin fecha: {registros_inicial - len(df)}")

      • Removidos sin fecha: 0


In [38]:
# Quitar registros sin ventas o ventas = 0
df = df[df['Ventas'].notna()].copy()
df = df[df['Ventas'] > 0].copy()
print(f"      • Removidos con ventas <= 0: {registros_inicial - len(df)}")

      • Removidos con ventas <= 0: 369


In [39]:
# Quitar fechas futuras (errores de carga)
fecha_hoy = pd.Timestamp.now()
df = df[df['Fecha_Limpia'] <= fecha_hoy].copy()

In [40]:
# 4. AGRUPAR POR DÍA
df_diario = df.groupby('Fecha_Limpia')['Ventas'].sum().reset_index()
df_diario

,Fecha_Limpia,Ventas
0,2022-01-01,11698.29
1,2022-01-02,9799.00
2,2022-01-03,7575.08
3,2022-01-04,8681.93
4,2022-01-05,8492.27
...,...,...
1090,2024-12-26,6014.61
1091,2024-12-27,7517.31
1092,2024-12-28,9336.62
1093,2024-12-29,7789.13


In [41]:
df_diario.columns = ['ds', 'y']
df_diario = df_diario.sort_values('ds').reset_index(drop = True)
df_diario

,ds,y
0,2022-01-01,11698.29
1,2022-01-02,9799.00
2,2022-01-03,7575.08
3,2022-01-04,8681.93
4,2022-01-05,8492.27
...,...,...
1090,2024-12-26,6014.61
1091,2024-12-27,7517.31
1092,2024-12-28,9336.62
1093,2024-12-29,7789.13


In [42]:
# 5. REMOVER OUTLIERS EXTREMOS
# Calcular límites usando IQR (Rango Intercuartílico)
Q1 = df_diario['y'].quantile(0.25)
Q3 = df_diario['y'].quantile(0.75)
IQR = Q3 - Q1
IQR

4284.975

In [64]:
Q1

7662.125

In [65]:
Q3

11947.1

In [43]:
# Límites: valores más de 3x IQR fuera del rango normal
limite_inferior = Q1 - 3 * IQR
limite_superior = Q3 + 3 * IQR

outliers_antes = len(df_diario)
outliers_antes

1095

In [44]:
df_diario = df_diario[
    (df_diario['y'] >= limite_inferior) & 
    (df_diario['y'] <= limite_superior)
].copy()

print(f"• Outliers removidos: {outliers_antes - len(df_diario)}")

• Outliers removidos: 0
• Límites: $-5,193 a $24,802


In [45]:
# 6. REMOVER DÍAS CON VENTAS ANORMALMENTE BAJAS (< 1% del promedio)
promedio = df_diario['y'].mean()
limite_minimo = promedio * 0.01  # 1% del promedio

dias_antes = len(df_diario)
df_diario = df_diario[df_diario['y'] >= limite_minimo].copy()
print(f"• Días con ventas < ${limite_minimo:.0f} removidos: {dias_antes - len(df_diario)}")

• Días con ventas < $101 removidos: 0


In [46]:
# 7. VALIDAR QUE TENEMOS SUFICIENTES DATOS
if len(df_diario) < 60:
    print(f"\n❌ ERROR: Solo quedan {len(df_diario)} días después de limpiar")
    print("   Se necesitan al menos 60 días para un modelo confiable")
    exit()

In [47]:
# 8. RELLENAR DÍAS FALTANTES CON INTERPOLACIÓN
# Crear rango completo de fechas
fecha_min = df_diario['ds'].min()
fecha_max = df_diario['ds'].max()
rango_completo = pd.date_range(start = fecha_min, end = fecha_max, freq = 'D')
rango_completo

DatetimeIndex(['2022-01-01', '2022-01-02', '2022-01-03', '2022-01-04',
               '2022-01-05', '2022-01-06', '2022-01-07', '2022-01-08',
               '2022-01-09', '2022-01-10',
               ...
               '2024-12-21', '2024-12-22', '2024-12-23', '2024-12-24',
               '2024-12-25', '2024-12-26', '2024-12-27', '2024-12-28',
               '2024-12-29', '2024-12-30'],
              dtype='datetime64[ns]', length=1095, freq='D')

In [48]:
# Reindexar y rellenar
df_diario = df_diario.set_index('ds').reindex(rango_completo)
df_diario.index.name = 'ds'
df_diario

,y
ds,
2022-01-01,11698.29
2022-01-02,9799.00
2022-01-03,7575.08
2022-01-04,8681.93
2022-01-05,8492.27
...,...
2024-12-26,6014.61
2024-12-27,7517.31
2024-12-28,9336.62


In [50]:
# Revertirlo porque ds no puede estar como índice al momento de llevarlo al modelo
df_diario = df_diario.reset_index()
df_diario.rename(columns = {'index':'ds'}, inplace = True)  # si tu índice se llamaba 'index'
df_diario['ds'] = pd.to_datetime(df_diario['ds'])

In [51]:
df_diario

,ds,y
0,2022-01-01,11698.29
1,2022-01-02,9799.00
2,2022-01-03,7575.08
3,2022-01-04,8681.93
4,2022-01-05,8492.27
...,...,...
1090,2024-12-26,6014.61
1091,2024-12-27,7517.31
1092,2024-12-28,9336.62
1093,2024-12-29,7789.13


### Crear y entrenar el modelo predictivo

In [52]:
modelo = Prophet(
    yearly_seasonality = True,
    weekly_seasonality = True,
    daily_seasonality = False,
    seasonality_mode = 'multiplicative',  # Mejor para datos con variación proporcional, otra opción es "additive"
    changepoint_prior_scale = 0.05,       # Menos sensible a cambios bruscos
    interval_width = 0.80                 # Intervalo de confianza del 80%
)

# Agregar festivos/eventos si hay patrones claros
# modelo.add_country_holidays(country_name = 'UY')  # Descomentar para incluir festivos urguayos

modelo.fit(df_diario)

print("✓ Modelo entrenado exitosamente\n")

12:22:35 - cmdstanpy - INFO - Chain [1] start processing
12:22:35 - cmdstanpy - INFO - Chain [1] done processing


   ✓ Modelo entrenado exitosamente



### Hacer las predicciones

In [54]:
futuro = modelo.make_future_dataframe(periods = 90, freq = 'D')
prediccion = modelo.predict(futuro)

# Asegurar que las predicciones sean positivas
prediccion['yhat'] = prediccion['yhat'].clip(lower = 0)
prediccion['yhat_lower'] = prediccion['yhat_lower'].clip(lower = 0)
prediccion['yhat_upper'] = prediccion['yhat_upper'].clip(lower = 0)

print(f"✓ Predicción completada\n")

✓ Predicción completada



### Preparar datos para llevar a Power BI

In [55]:
resultado = prediccion[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].copy()
resultado.columns = ['Fecha', 'Ventas_Predichas', 'Ventas_Min', 'Ventas_Max']

resultado['Ventas_Predichas'] = resultado['Ventas_Predichas'].round(2)
resultado['Ventas_Min'] = resultado['Ventas_Min'].round(2)
resultado['Ventas_Max'] = resultado['Ventas_Max'].round(2)

resultado['Tipo'] = 'Histórico'
resultado.loc[resultado['Fecha'] > df_diario['ds'].max(), 'Tipo'] = 'Predicción'

print(f"✓ Datos preparados\n")

✓ Datos preparados



In [56]:
resultado

,Fecha,Ventas_Predichas,Ventas_Min,Ventas_Max,Tipo
0,2022-01-01,11664.31,9848.83,13344.45,Histórico
1,2022-01-02,9361.16,7687.27,11016.52,Histórico
2,2022-01-03,7246.12,5527.50,8926.32,Histórico
3,2022-01-04,6987.93,5256.39,8756.06,Histórico
4,2022-01-05,6828.94,5041.87,8481.45,Histórico
...,...,...,...,...,...
1180,2025-03-26,10031.16,8340.94,11748.74,Predicción
1181,2025-03-27,10048.53,8376.02,11864.51,Predicción
1182,2025-03-28,12370.18,10564.87,14187.30,Predicción
1183,2025-03-29,14890.42,13181.60,16600.18,Predicción


### Validar Modelo

MAPE = Mean Absolute Percentage Error - Error Porcentual Absoluto Medio.

Es una métrica que mide, en porcentaje, cuánto se equivoca un modelo en promedio respecto a los valores reales.

Mientras más bajo, mejor el modelo.

In [57]:
# Comparar solo datos históricos
historico = resultado[resultado['Tipo'] == 'Histórico'].copy()
historico = historico.merge(df_diario, left_on = 'Fecha', right_on = 'ds', how = 'inner')
historico

,Fecha,Ventas_Predichas,Ventas_Min,Ventas_Max,Tipo,ds,y
0,2022-01-01,11664.31,9848.83,13344.45,Histórico,2022-01-01,11698.29
1,2022-01-02,9361.16,7687.27,11016.52,Histórico,2022-01-02,9799.00
2,2022-01-03,7246.12,5527.50,8926.32,Histórico,2022-01-03,7575.08
3,2022-01-04,6987.93,5256.39,8756.06,Histórico,2022-01-04,8681.93
4,2022-01-05,6828.94,5041.87,8481.45,Histórico,2022-01-05,8492.27
...,...,...,...,...,...,...,...
1090,2024-12-26,9623.42,7851.91,11354.08,Histórico,2024-12-26,6014.61
1091,2024-12-27,11804.79,9950.63,13530.00,Histórico,2024-12-27,7517.31
1092,2024-12-28,14155.00,12382.23,15770.50,Histórico,2024-12-28,9336.62
1093,2024-12-29,11987.55,10247.98,13575.97,Histórico,2024-12-29,7789.13


In [58]:
mae = mean_absolute_error(historico['y'], historico['Ventas_Predichas'])

# Calcular MAPE manualmente (evitando división por valores muy pequeños)
historico['ape'] = np.abs((historico['y'] - historico['Ventas_Predichas']) / historico['y'])

In [59]:
historico

,Fecha,Ventas_Predichas,Ventas_Min,Ventas_Max,Tipo,ds,y,ape
0,2022-01-01,11664.31,9848.83,13344.45,Histórico,2022-01-01,11698.29,0.002905
1,2022-01-02,9361.16,7687.27,11016.52,Histórico,2022-01-02,9799.00,0.044682
2,2022-01-03,7246.12,5527.50,8926.32,Histórico,2022-01-03,7575.08,0.043427
3,2022-01-04,6987.93,5256.39,8756.06,Histórico,2022-01-04,8681.93,0.195118
4,2022-01-05,6828.94,5041.87,8481.45,Histórico,2022-01-05,8492.27,0.195864
...,...,...,...,...,...,...,...,...
1090,2024-12-26,9623.42,7851.91,11354.08,Histórico,2024-12-26,6014.61,0.600007
1091,2024-12-27,11804.79,9950.63,13530.00,Histórico,2024-12-27,7517.31,0.570348
1092,2024-12-28,14155.00,12382.23,15770.50,Histórico,2024-12-28,9336.62,0.516073
1093,2024-12-29,11987.55,10247.98,13575.97,Histórico,2024-12-29,7789.13,0.539010


### Quitar valores donde APE es infinito o muy grande

Evita que valores extremos (como divisiones por cero o picos atípicos) distorsionen el MAPE, que de otra forma sería muy alto e irrelevante.


#### Valores cercanos a cero

Si algún y es muy pequeño o cero, APE explota → filtrar valores grandes

#### Sesgo al filtrar

Al eliminar errores grandes, el MAPE resultante no refleja todos los errores, solo los “aceptables”.

Útil para análisis de desempeño general, pero no indica lo peor que puede fallar el modelo.

#### Alternativas

Para datos con muchos ceros o picos, se puede usar SMAPE (Symmetric MAPE) o MAPE truncado.
SMAPE evita dividir por valores cercanos a cero.

In [61]:
historico_filtrado = historico[historico['ape'] < 2]  # Quitar errores > 200%
mape = historico_filtrado['ape'].mean() * 100

print(f"\n📊 Métricas de precisión:")
print(f"• Error absoluto promedio: ${mae:,.2f}")
print(f"• Error porcentual promedio: {mape:.2f}%")

if mape < 15:
    print(f"      ✅ EXCELENTE - Modelo muy confiable")
elif mape < 25:
    print(f"      ✅ BUENO - Modelo confiable para planificación")
elif mape < 40:
    print(f"      ⚠️  REGULAR - Usar con precaución")
else:
    print(f"      ⚠️  BAJO - Considerar solo como referencia")

print()


📊 Métricas de precisión:
• Error absoluto promedio: $959.04
• Error porcentual promedio: 9.90%
      ✅ EXCELENTE - Modelo muy confiable



### Exportar datos a CSV

In [62]:
nombre_archivo = 'prediccion_ventas_90dias.csv'
resultado.to_csv(nombre_archivo, index = False, encoding = 'utf-8-sig')

print(f"✅ Archivo guardado: {nombre_archivo}\n")

✅ Archivo guardado: prediccion_ventas_90dias.csv



### Resumen Final

In [63]:
print("="*70)
print("✅ PROCESO COMPLETADO")
print("="*70)

historico_result = resultado[resultado['Tipo'] == 'Histórico']
futuro_result = resultado[resultado['Tipo'] == 'Predicción']

print(f"\n📊 RESUMEN:")
print(f"   • Días históricos: {len(historico_result)}")
print(f"   • Días predichos: {len(futuro_result)}")
print(f"   • Ventas promedio históricas: ${historico_result['Ventas_Predichas'].mean():,.2f}")
print(f"   • Ventas promedio predichas: ${futuro_result['Ventas_Predichas'].mean():,.2f}")

variacion = ((futuro_result['Ventas_Predichas'].mean() / historico_result['Ventas_Predichas'].mean()) - 1) * 100
print(f"   • Variación esperada: {variacion:+.1f}%")

print(f"\n📈 PRÓXIMOS 7 DÍAS PREDICHOS:")
print(futuro_result.head(7)[['Fecha', 'Ventas_Predichas']].to_string(index=False))

print(f"\n🎯 SIGUIENTE PASO EN POWER BI:")
print(f"   1. Get Data → Text/CSV → {nombre_archivo}")
print(f"   2. Crear gráfico de líneas:")
print(f"      • Eje X: Fecha")
print(f"      • Valores: Ventas_Predichas")
print(f"      • Leyenda: Tipo (Histórico vs Predicción)")
print(f"   3. Agregar banda de confianza (Ventas_Min y Ventas_Max)")

print("\n" + "="*70)

✅ PROCESO COMPLETADO

📊 RESUMEN:
   • Días históricos: 1095
   • Días predichos: 90
   • Ventas promedio históricas: $10,136.69
   • Ventas promedio predichas: $9,465.15
   • Variación esperada: -6.6%

📈 PRÓXIMOS 7 DÍAS PREDICHOS:
     Fecha  Ventas_Predichas
2024-12-31           9187.10
2025-01-01           9620.22
2025-01-02           8839.59
2025-01-03          10901.25
2025-01-04          13145.26
2025-01-05          10880.59
2025-01-06           9669.77

🎯 SIGUIENTE PASO EN POWER BI:
   1. Get Data → Text/CSV → prediccion_ventas_90dias.csv
   2. Crear gráfico de líneas:
      • Eje X: Fecha
      • Valores: Ventas_Predichas
      • Leyenda: Tipo (Histórico vs Predicción)
   3. Agregar banda de confianza (Ventas_Min y Ventas_Max)

